In [1]:
import json
import pandas as pd
from yome import Session
from yome.models import *
from yome.util import to_df
import re

In [2]:
from sqlalchemy import exists
from sqlalchemy.orm import aliased

# Look for genes that have absolutely no information

starting with EcoCyc phrase "No information about this"

In [3]:
with open('../ecocyc/results.json') as f:
    results = json.load(f)
df = pd.DataFrame(results)

FileNotFoundError: [Errno 2] No such file or directory: '../ecocyc/results.json'

In [ ]:
session = Session()

# aliases
KnowledgebaseGene2 = aliased(KnowledgebaseGene)
Knowledgebase2 = aliased(Knowledgebase)
KnowledgebaseFeature2 = aliased(KnowledgebaseFeature)

# Genes with no info in EcoCyc
no_info = (
    session.query(Gene.locus_id.label('gene_locus_id'), 
                  KnowledgebaseGene.id.label('knowledgebase_gene_id'), 
                  Knowledgebase, 
                  KnowledgebaseFeature.feature)
    .join(KnowledgebaseGene)
    .join(Knowledgebase)
    .join(KnowledgebaseFeature)
    .filter(Knowledgebase.name == 'EcoCyc')
    .filter(KnowledgebaseFeature.feature_type == 'summary_html')
    .filter(KnowledgebaseFeature.feature.like('%No information about this%'))
)
# Make a subquery
no_info_sub = no_info.subquery('no_info_sub')

# Genes that do not start with y in EcoCyc and have no information
no_info_not_start_y = (
    no_info.filter(~KnowledgebaseGene.primary_name.like('y%'))
)

# Genes with no info in EcoCyc that are ranked "high" in another db
no_info_and_high = (
    session.query(Gene.locus_id,
                  KnowledgebaseGene.id,
                  KnowledgebaseGene2.annotation_quality,
                  Knowledgebase2.name)
    .filter(Gene.locus_id == no_info_sub.c.gene_locus_id)
    .filter(KnowledgebaseGene.id == no_info_sub.c.knowledgebase_gene_id)
    .join(KnowledgebaseGene2, KnowledgebaseGene2.gene_id == Gene.id)
    .join(Knowledgebase2, Knowledgebase2.id == KnowledgebaseGene2.knowledgebase_id)
    .filter(KnowledgebaseGene2.annotation_quality != 'low')
    .distinct(KnowledgebaseGene.id)
)

# Genes with no info in EcoCyc that are not ranked "high" in another db
no_info_and_not_high = (
    no_info
    .filter(
        ~session.query(KnowledgebaseGene2, Knowledgebase2)
        .filter(KnowledgebaseGene2.gene_id == Gene.id)
        .join(Knowledgebase2, Knowledgebase2.id == KnowledgebaseGene2.knowledgebase_id)
        .filter(KnowledgebaseGene2.annotation_quality != 'low')
        .filter(Knowledgebase2.name != 'EcoCyc')
        .exists()
    )
)

# Examples of no_info_and_not_high
no_info_and_not_high_examples = (
    session.query(Gene.locus_id,
                  KnowledgebaseGene.id,
                  KnowledgebaseGene2.primary_name,
                  KnowledgebaseGene2.annotation_quality,
                  Knowledgebase2.name,
                  KnowledgebaseFeature2.feature)
    .filter(Gene.locus_id == no_info_sub.c.gene_locus_id)
    .filter(KnowledgebaseGene.id == no_info_sub.c.knowledgebase_gene_id)
    .join(KnowledgebaseGene2, KnowledgebaseGene2.gene_id == Gene.id)
    .join(Knowledgebase2, Knowledgebase2.id == KnowledgebaseGene2.knowledgebase_id)
    .join(KnowledgebaseFeature2, KnowledgebaseFeature2.knowledgebase_gene_id == KnowledgebaseGene2.id)
    .filter(KnowledgebaseGene2.annotation_quality == 'low')
    .filter(Knowledgebase2.name != 'EcoCyc')
    .filter(Knowledgebase2.name != 'Y-ome')
)

session.close()

In [ ]:
no_info.count(), no_info_not_start_y.count(), no_info_and_high.count(), no_info_and_not_high.count()

In [ ]:
df = to_df(no_info_and_not_high_examples, ['locus_id', 'ig1', 'name', 'annotation_quality', 'kbase', 'feature']).drop('ig1', axis=1)

## Filter out phrases for other databases

In [ ]:
phrases = [
    ('EcoGene', r'.* family protein$'),
    ('EcoGene', r'.* family\.?$'),
    ('EcoGene', r'Null'),
    ('EcoGene', r'Putative enzyme; Not classified'),
    ('EcoGene', r'Phenotype; Not classified'),
    ('EcoGene', r'Putative factor; Not classified'),
    ('EcoGene', r'.*uncharacterized protein.*'),
    ('EcoGene', r'.*pseudogene.*'),
    ('EcoGene', r'.*function unknown.*'),
    ('EcoGene', r'.*Phage or Prophage Related.*'),
    ('UniProt', r'^$'),
    ('UniProt', r'Chain \(1\)'),
    ('UniProt', r'^\d\.0$'),
    ('UniProt', r'^Putative uncharacterized'),
    ('UniProt', r'^Putative protein'),
    ('UniProt', r'^Uncharacterized protein'),
    ('UniProt', r'^Protein .{4}$'),
    ('UniProt', r'^UPF\d{4} Protein .{4}$'),
    ('UniProt', r'FUNCTION: Not yet known\.'),
]
df_filtered = df
for kbase, reg in phrases:
    df_filtered = df_filtered[~((df_filtered.kbase == kbase) & df_filtered.feature.str.match(reg, flags=re.IGNORECASE))]

In [ ]:
df_filtered.to_csv('no-ecocyc-info-other-db-annotations.tsv', sep='\t')

In [ ]:
# Find no info genes that are not in the filtered list
session = Session()
no_info_not_in_filtered = (
    no_info
    .filter(Gene.locus_id.notin_(df_filtered.loc[:, 'locus_id']))
)
session.close()

In [ ]:
# double check numbers add up
assert(no_info.count() == no_info_not_in_filtered.count() + len(df_filtered.locus_id.unique()))

## These are the genes that really have no hints at all:

In [ ]:
no_info_not_in_filtered.count()

## Save a list of these with primary names

In [ ]:
session = Session()
primary_names = {x[0]: x[1] for x in (
    session.query(Gene.locus_id, KnowledgebaseGene.primary_name)
    .join(KnowledgebaseGene)
    .distinct(Gene.id)
    .all()
)}
session.close()
def get_primary_name(b_num):
    return primary_names[b_num]

In [ ]:
no_info_not_in_filtered_names = to_df(no_info_not_in_filtered, cols=['locus_id'])
no_info_not_in_filtered_names.loc[:, 'name'] = no_info_not_in_filtered_names.locus_id.map(get_primary_name)

In [ ]:
no_info_not_in_filtered_names.to_csv('no_info_not_in_filtered_names.tsv', sep='\t')